###Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

####Importing Dataset

In [3]:
movies = pd.read_csv('drive/MyDrive/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')

users = pd.read_csv('drive/MyDrive/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')

ratings = pd.read_csv('drive/MyDrive/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')

###Preparing the Training and Test Set

In [4]:
training_set = pd.read_csv('drive/MyDrive/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('drive/MyDrive/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

###Getting the number of users and movies

In [5]:
 total_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
 total_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
 print("total users: ", total_users)
 print("total movies: ", total_movies)

total users:  943
total movies:  1682


###Converting the data into an array with users in lines and movies in columns

In [6]:
def convert(dataset):
  user_movies = []
  for user_id in range(1,total_users + 1):
    movie_ids = dataset[:,1][dataset[:,0] == user_id]
    rating_ids = dataset[:,2][dataset[:,0] == user_id]
    rating_list = np.zeros(total_movies)
    rating_list[movie_ids - 1] = rating_ids
    user_movies.append(list(rating_list))
  return user_movies

In [7]:
training_set = convert(training_set)
test_set = convert(test_set)

###Convert the data into Torch tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

###Creating the architecture of Neural Network

In [15]:
class AE(nn.Module):
  def __init__(self, ):
    super(AE, self).__init__()
    self.full_connection1 = nn.Linear(total_movies, 20) # 20 nodes in first hidden layer
    self.full_connection2 = nn.Linear(20, 10)
    self.full_connection3 = nn.Linear(10, 20)
    self.full_connection4 = nn.Linear(20, total_movies)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.full_connection1(x))
    x = self.activation(self.full_connection2(x))
    x = self.activation(self.full_connection3(x))
    x = self.full_connection4(x)
    return x

In [16]:
ae = AE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(ae.parameters(), lr = 0.01, weight_decay = 0.5)

###Training the AE

In [20]:
epochs = 200 
for e in range(1, epochs + 1):
  train_loss = 0
  counter = 0.
  for user_id in range(total_users):
    input = Variable(training_set[user_id]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      #atleast one rating which is > 0
      predicted_rating = ae(input)
      target.required_grad = False
      predicted_rating[target == 0] = 0
      loss = criterion(predicted_rating, target)
      mean_corrector = total_movies / float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      counter += 1.
      optimizer.step()

  print('epoch: ' + str(e) + ' loss: ' + str(train_loss/counter))

epoch: 1 loss: tensor(1.7605)
epoch: 2 loss: tensor(1.0963)
epoch: 3 loss: tensor(1.0532)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0311)
epoch: 6 loss: tensor(1.0263)
epoch: 7 loss: tensor(1.0239)
epoch: 8 loss: tensor(1.0221)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0197)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0181)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0177)
epoch: 15 loss: tensor(1.0171)
epoch: 16 loss: tensor(1.0170)
epoch: 17 loss: tensor(1.0166)
epoch: 18 loss: tensor(1.0165)
epoch: 19 loss: tensor(1.0164)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0161)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0159)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0131)
epoch: 30 loss: tensor(1.0121)
epoch: 31 loss: tensor(1.0097)
epoch: 32 loss: tensor(1.0100)
epoch: 33 loss: t

###Testing the AE

In [24]:
test_loss = 0
counter = 0.
for user_id in range(total_users):
  input = Variable(training_set[user_id]).unsqueeze(0)
  target = Variable(test_set[user_id]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    #atleast one rating which is > 0
    predicted_rating = ae(input)
    target.required_grad = False
    predicted_rating[target == 0] = 0
    loss = criterion(predicted_rating, target)
    mean_corrector = total_movies / float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    counter += 1.

print('test loss: ' + str(test_loss/counter))

test loss: tensor(0.9277)
